In [2]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from pprint import pprint
# import csv
import json
import datetime
import unicodecsv as csv

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


In [3]:
def get_creds():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return creds

In [4]:
def get_message(msg_id, service):
    # message = service.users().messages().get(userId='me', id=msg_id).execute()
    # pprint(message.keys())
    # pprint(message)
    #
    # print('-'*100)
    # date, from, subject

    hs = ['Date', 'From', 'Subject', 'X-Unsubscribe-Web']
    message = service.users().messages().get(userId='me', id=msg_id, format='metadata', metadataHeaders=hs).execute()

    # pprint(message)
    return message

    # message = service.users().messages().get(userId='me', id=msg_id, format='minimal' ).execute()
    # pprint( message)

    # message = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    # pprint( message)
    # print(msg_id,message['historyId'],message['internalDate'],message['sizeEstimate'],message['labelIds'],message['snippet'])


In [5]:
def get_messages_by_page(service):
    label = ['CATEGORY_FORUMS']
    maxResult = 100
    t = ['01650191948800808238', '07460280352673579117']
    for tt in t:
        messages = service.users().messages().list(userId='me', labelIds=label, maxResults=maxResult,
                                                   pageToken=t).execute()
        pprint(messages)

In [6]:
def user_messages(service, labels):
    labels = ['CATEGORY_PROMOTIONS']
    print('labels: ', labels)
    maxResult = 100
    for label in labels:
        print('getting emails with label:', label)
        count = 0
        msgId_list = []
        msg_list = []
        messages = service.users().messages().list(userId='me', labelIds=label, maxResults=maxResult ).execute()
        count += len(messages)
        # print(label, 'got', len(messages['messages']), 'of', messages['resultSizeEstimate'], 'messages')

        # open writer to write each msg as processed

        curr_time = datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d%H%M%S')
        with open(label + curr_time + '.csv', 'w') as out_file:
            msg_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, encoding='utf-8')
            header = ['id', 'threadId', 'historyId', 'internalDate', 'sizeEstimate', 'labelIds', 'from', 'subject',
                      'snippet']
            msg_writer.writerow(header)

            for msgId in messages['messages']:
                msgId_list.append(str(msgId['id']))
                # print('getting message for email id:', msgId['id'])
                msg = get_message(str(msgId['id']), service)


                from_who = msg['payload']['headers'][1]['value']
                subject = msg['payload']['headers'][2]['value']
                msg_writer.writerow(
                    [msg['id'], msg['threadId'], msg['historyId'], msg['internalDate'], msg['sizeEstimate'],
                     msg['labelIds'], from_who, subject,
                     msg['snippet']])

            print('nextPageToken:', messages['nextPageToken'])

            if 'nextPageToken' in messages:
                more = True
                nextPageToken = messages['nextPageToken']

            while more:
                messages = service.users().messages().list(userId='me', labelIds=label, maxResults=maxResult,
                                                           pageToken=nextPageToken).execute()
                count += len(messages)

                for msgId in messages['messages']:
                    # msgId_list.append(str(msgId['id']))
                    # print('getting message for email id:', str(msgId['id']))
                    msg = get_message(str(msgId['id']), service)
                    from_who = msg['payload']['headers'][1]['value']
                    subject = msg['payload']['headers'][2]['value']
                    msg_writer.writerow(
                        [msg['id'], msg['threadId'], msg['historyId'], msg['internalDate'], msg['sizeEstimate'],
                         msg['labelIds'], from_who, subject,
                         msg['snippet']])

                print('nextPageToken:', messages['nextPageToken'])
                if 'nextPageToken' in messages:
                    nextPageToken = messages['nextPageToken']
                else:
                    more = False

                print(count)

        # processed_so_far = 0
        # for msg in messages['messages']:
        #     processed_so_far +=1
        #     if processed_so_far %10 ==0 :
        #         break
        #         # print(processed_so_far)
        #     get_message(msg['id'], service)




In [7]:
def list_user_labels(service):
    # Call the Gmail API
    labels = service.users().labels().list(userId='me').execute()
    # pprint(labels['labels'])
    label_list = []
    # print('you have the following labels:')
    for label in labels['labels']:
        # print(label['type'], label['name'])
        if str(label['name']) != 'Junk' or str(label['name']) != 'Notes':
            label_list.append(str(label['name']))
    # print(label_list)
    return label_list

    #
    # labels = results.get('labels', [])
    #
    # if not labels:
    #     print('No labels found.')
    # else:
    #     print('Labels:')
    #     for label in labels:
    #         print(label['name'])


def list_user_info(service):
    print('user info:')
    profile = service.users().getProfile(userId='me').execute()
    # print(profile)
    pprint(profile)


def user_drafts(service):
    print('user draft:')
    drafts = service.users().drafts().list(userId='me').execute()
    # print(profile)
    # drafts = {drafts:draft[], resultSizeEstimate: 37  }
    print('You have', drafts['resultSizeEstimate'], 'drafts')

    # pprint(drafts['drafts'])
    # [{u'id': u'r6462568432385231994',
    #   u'message': {u'id': u'16b8a800776af1d5', u'threadId': u'16b8a7feffabb463'}},

    # draft = service.users().drafts().get(userId='me', id='r6462568432385231994').execute()
    # pprint(draft)
    # pprint(draft['message'].keys())
    # pprint(draft)




In [8]:

# def main():
#     creds = get_creds()
#     service = build('gmail', 'v1', credentials=creds)

#     list_user_info(service)

#     labels = list_user_labels(service)

#     user_messages(service, labels)

#     # get_message('16c594bc00becce9',service)

#     # get_messages_by_page(service)
    
# if __name__ == '__main__':
#     main()

# # service.users().*
# # * can be any end point,

In [9]:
creds = get_creds()
service = build('gmail', 'v1', credentials=creds)
list_user_info(service)

user info:
{u'emailAddress': u'wangqiman0111@gmail.com',
 u'historyId': u'10820981',
 u'messagesTotal': 44641,
 u'threadsTotal': 41110}


In [10]:
labels = list_user_labels(service)
labels
#     # get_messages_by_page(service)

['CATEGORY_PERSONAL',
 'CATEGORY_SOCIAL',
 'Junk',
 'Notes',
 'CATEGORY_FORUMS',
 'IMPORTANT',
 'wonton',
 'CATEGORY_UPDATES',
 'CHAT',
 'SENT',
 'INBOX',
 'TRASH',
 'CATEGORY_PROMOTIONS',
 'DRAFT',
 'SPAM',
 'STARRED',
 'UNREAD']

In [27]:
get_message(,service)

{u'historyId': u'10749465',
 u'id': u'16c72d025fdf7c1d',
 u'internalDate': u'1565294337000',
 u'labelIds': [u'CATEGORY_SOCIAL', u'INBOX'],
 u'payload': {u'headers': [{u'name': u'From',
    u'value': u'Monica Nagar <invitations@linkedin.com>'},
   {u'name': u'Subject',
    u'value': u'Qiman, please add me to your LinkedIn network'},
   {u'name': u'Date', u'value': u'Thu, 8 Aug 2019 19:58:57 +0000 (UTC)'}],
  u'mimeType': u'multipart/alternative'},
 u'sizeEstimate': 32147,
 u'snippet': u'Hi Qiman, I&#39;d like to join your LinkedIn network. LinkedIn Qiman Wang Hi Qiman, I&#39;d like to join your LinkedIn network. Monica Nagar PM/ Sr Recruiter - US Citizen Greater Seattle Area Accept',
 u'threadId': u'16c70cd5e1834a51'}

In [29]:
msg = get_message('16c72d025fdf7c1d', service)


from_who = msg['payload']['headers'][1]['value']
subject = msg['payload']['headers'][2]['value']
list(
    [msg['id'], msg['threadId'], msg['historyId'], msg['internalDate'], msg['sizeEstimate'],
     msg['labelIds'], from_who, subject,
     msg['snippet']])


[u'16c72d025fdf7c1d',
 u'16c70cd5e1834a51',
 u'10749465',
 u'1565294337000',
 32147,
 [u'CATEGORY_SOCIAL', u'INBOX'],
 u'Qiman, please add me to your LinkedIn network',
 u'Thu, 8 Aug 2019 19:58:57 +0000 (UTC)',
 u'Hi Qiman, I&#39;d like to join your LinkedIn network. LinkedIn Qiman Wang Hi Qiman, I&#39;d like to join your LinkedIn network. Monica Nagar PM/ Sr Recruiter - US Citizen Greater Seattle Area Accept']

In [30]:
msg['payload']

{u'headers': [{u'name': u'From',
   u'value': u'Monica Nagar <invitations@linkedin.com>'},
  {u'name': u'Subject',
   u'value': u'Qiman, please add me to your LinkedIn network'},
  {u'name': u'Date', u'value': u'Thu, 8 Aug 2019 19:58:57 +0000 (UTC)'}],
 u'mimeType': u'multipart/alternative'}

In [36]:
for kv in msg['payload']['headers']:
    print(kv)
    if kv['name']== 'From':
        print(kv['value'])

{u'name': u'From', u'value': u'Monica Nagar <invitations@linkedin.com>'}
Monica Nagar <invitations@linkedin.com>
{u'name': u'Subject', u'value': u'Qiman, please add me to your LinkedIn network'}
{u'name': u'Date', u'value': u'Thu, 8 Aug 2019 19:58:57 +0000 (UTC)'}
